# Actions

These are _action_ functions used in other notebooks.

## Reusing Actions

These will typically be used with the `doit` callable style.

```python
with importnb.Notebook():
    from _actions import some_action

def some_task():
    return dict(
        actions=[
            (some_action, [some_positional_arg], {some_named="arg"})
        ]
    )
```

## Conventions

This notebook should limit its imports.

In [ ]:
import hashlib
import pprint
import shutil
import tempfile
import urllib.request
from pathlib import Path

import doit

Each should return `True` or `None` if successful, and avoid keeping internal, hidden state.

In [ ]:
Boolish = None | bool

## The Actions

Run a function, pre-announcing what will run.

> copy a file or folder

In [ ]:
def copy(src: Path, dest: Path) -> Boolish:
    clean(dest)
    if not dest.parent.exists():
        dest.parent.mkdir(parents=true)
    if src.is_dir():
        shutil.copytree(src, dest)
    else:
        shutil.copy2(src, dest)

> delete some files or folders

In [ ]:
def clean(*paths: Path, globs: dict[Path, list[str]] = None) -> None:
    if globs:
        for root, root_globs in globs.items():
            for glob in root_globs:
                paths = [*paths, *sorted(root.glob(glob))]
    for path in paths:
        if path.is_dir():
            shutil.rmtree(path)
        elif path.exists():
            path.unlink()

In [ ]:
class AnnouncingCmdAction(doit.action.CmdAction):
    def execute(self, out=None, err=None):
        try:
            action = self.expand_action()
            # set environ to change output buffering
            kwargs = self.pkwargs.copy()
            print(">>> in", kwargs["cwd"])
            print("\t", *action, file=out, flush=True)
        except Exception:
            pass

        return super().execute(out=out, err=err)

Run a command in a directory.

In [ ]:
def run(args, kwargs):
    if "cwd" not in kwargs:
        raise ValueError(f"... a `cwd` is required for {args}")
    return AnnouncingCmdAction(list(map(str, args)), shell=False, **kwargs)

Run `git` in a directory.

In [ ]:
def git(args, kwargs):
    return run(["git", *args], kwargs)

In [ ]:
def sha256_some(shasums: Path, root: Path, patterns: list[str]) -> Boolish:
    entries = {}
    for pattern in patterns:
        for p in root.glob(pattern):
            if p == shasums:
                continue
            entries[p.name] = hashlib.sha256(p.read_bytes()).hexdigest()
    lines = [f"{sh}  {p}" for p, sh in sorted(entries.items())]
    if not lines:
        raise ValueError(f"!!! nothing to hash for {shasums}: {patterns}")
    output = "\n".join(lines)
    print(output)
    shasums.parent.mkdir(exist_ok=True, parents=True)
    shasums.write_text(output, encoding="utf-8")

In [ ]:
def resolve_globbish(cwd, globbishes):
    resolved = []
    for globbish in globbishes:
        if callable(globbish):
            resolved += sorted(globbish(cwd))
        elif isinstance(globbish, str) and "*" in globbish:
            resolved += [p for p in cwd.glob(globbish) if not p.is_dir()]
        elif isinstance(globbish, str):
            resolved += [cwd / globbish]
        elif hasattr(globbish, "glob_neighbor"):
            glob_neighbor = globbish.glob_neighbor
            if glob_neighbor:
                pattern, neighbor = glob_neighbor
                for found in resolve_globbish(cwd, [pattern]):
                    if "{" in neighbor:
                        resolved += [
                            (found / neighbor.format(stem=found.stem)).resolve(),
                        ]
                    else:
                        resolved += [(found / neighbor).resolve()]
        else:
            resolved += [globbish]
    return sorted(set(resolved))

In [ ]:
def replace_between(src: Path, dest: Path, delimiter: str) -> Boolish:
    src_text = src.read_text(encoding="utf-8")
    dest_text = dest.read_text(encoding="utf-8")
    src_chunks = src_text.split(delimiter)
    dest_chunks = dest_text.split(delimiter)
    dest_text = delimiter.join([dest_chunks[0], src_chunks[1], dest_chunks[2]])
    dest.write_text(dest_text, encoding="utf-8")

In [ ]:
def transform_one(src: Path, dest: Path):
    import json

    import tomli_w
    import yaml

    try:
        import tomllib
    except ImportError:
        pass

    src_text = src.read_text(encoding="utf-8")

    if src.name.endswith("json"):
        data = json.loads(src_text)
    elif src.name.endswith("toml"):
        data = tomli.loads(src_text)
    elif src.name.endswith("yaml") or src.name.endswith("yml"):
        data = yaml.safe_load(src_text)
    else:
        return False

    if dest.name.endswith("json"):
        dest_text = json.dumps(data, indent=2)
    elif dest.name.endswith("toml"):
        dest_text = tomli_w.dumps(data)
    elif dest.name.endswith("yaml") or dest.name.endswith("yml"):
        dest_text = yaml.safe_dump(data, default_flow_style=False, sort_keys=False)
    else:
        return False
    dest.parent.mkdir(exist_ok=True, parents=True)
    dest.write_text(dest_text, encoding="utf-8")

In [ ]:
def get_parser_dumper(path: Path):
    if path.name.endswith("toml"):
        try:
            import tomllib
        except:
            import tomli as tomllib
        import tomli_w

        return tomllib.loads, tomli_w.dumps

    if path.name.endswith("json"):
        import json

        return json.loads, lambda data, **kwargs: json.dumps(data, **kwargs, indent=2)

    if path.name.endswith("yaml") or path.name.endswith(toml):
        import yaml

        return yaml.safe_load, lambda data, **kwargs: yaml.safe_dump(
            data, default_flow_style=False, sort_keys=False
        )

    raise ValueError(f"no parser for {path}")

In [ ]:
def validate_one(schema_path: Path, instance_path: Path | None = None) -> Boolish:
    import jsonschema

    parse_schema = get_parser_dumper(schema_path)[0]
    schema = parse_schema(schema_path.read_text(encoding="utf-8"))
    jsonschema.Draft7Validator.check_schema(schema)
    validator = jsonschema.Draft7Validator(
        schema, format_checker=jsonschema.Draft7Validator.format_checker
    )
    if instance_path:
        parse_instance = get_parser_dumper(instance_path)[0]
        instance = parse_instance(instance_path.read_text(encoding="utf-8"))
        errors = [*validator.iter_errors(instance)]
        if errors:
            pprint.pprint(errors)
            return False

In [ ]:
def fetch_one(self, url, dest):
    if dest.exists():
        self.log.info(f"already downloaded {dest.name}, skipping...")
        return

    if not dest.parent.exists():
        dest.parent.mkdir(parents=True)

    with tempfile.TemporaryDirectory() as td:
        tdp = Path(td)
        with urllib.request.urlopen(url) as response:  # noqa: S310
            tmp_dest = tdp / dest.name
            with tmp_dest.open("wb") as fd:
                shutil.copyfileobj(response, fd)
            last_modified = response.headers.get("Last-Modified")
            if last_modified:
                epoch_time = time.mktime(email.utils.parsedate(last_modified))
                os.utime(tmp_dest, (epoch_time, epoch_time))
        shutil.copy2(tmp_dest, dest)

In [ ]:
def extract_one(archive: Path, dest: Path):
    """Extract the contents of an archive to a path."""
    import libarchive

    clean(dest)

    dest.mkdir(parents=True)

    old_cwd = os.getcwd()
    os.chdir(str(dest))
    try:
        libarchive.extract_file(str(archive))
    finally:
        os.chdir(old_cwd)